# Working With Activities, Exercises, ITQs and SAQs

When creating active teaching and learning materials, an important learning design approach for prompting engagement is the inclusion of activities, where a call to action is provided and a learner is encouraged to perform a particular task or check their understanding in a particular way.

A large number of potentially reusable activities are defined in a structured way in OU-XML documents. If we are to reuse such activities, whether directly, with modification, or simply as a source of inspiration, we need to be able to browse or discover them in some way.

Currently, discovery is likely to arise from an academic remembering a particular activity from a particular module then sarching the VLE for that module, and searching within the VLE for the activity they remember. How much easier it would be if they could simply search over all activities, perhaps filtering down by module code, and then previewing the results in a meaningful way?

The `<Activity>`, `<Exercise>`, `<ITQ>` and `<SAQ>` elements all have a similar internal structure and only differ in the parent tag [[example docs](https://learn3.open.ac.uk/mod/oucontent/view.php?id=185747&section=8.1.1)].

Each element __must__ include a `<Question>` and may include a `<Heading>` and a `<Timing>` element; various different response elements may be provided (one or more of `<MediaContent>`, `<Interaction>`, `<Answer>`, `<Discussion>`) either as a single response, or within a `<Part>` tag inside a `<Multipart>` response element..

For example, at its simplest, an ITQ might take the following form:

```xml
<ITQ xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
    <Question>
        <Paragraph>In session 1, you met charged versions of atoms; what are they called? </Paragraph>
    </Question>
    <Answer>
        <Paragraph>These are ions (positively charged ions are called cations and negatively charged ions are called anions). </Paragraph>
    </Answer>
</ITQ>
```

ITQs may also be defined as multipart questions.

*Work in Progress*; the <MediaElement> enclosure within an activity element is not currently handled, although *all* media elements are extracted, irrespective of their context, in the generic section on extracting media items from OU-XML. *In the `Activity` context, it would probably make sense to use a foreign key relationship to provide a reference into the `media` table from an `activities` table.*

## Preparing the Ground

As ever, we need to set up a database connection:

In [1]:
from sqlite_utils import Database
import pandas as pd

# Open database connection
xml_dbname = "all_openlean_xml.db"
xml_db = Database(xml_dbname)

activity_dbname = "openlean_assets.db"
db = Database(activity_dbname)

And get a sample XML file, selecting one that we know contains structurally marked up glossary items:

In [2]:
from lxml import etree
import pandas as pd
from xml_utils import unpack, flatten

# Grab an OU-XML file that is known to contain activity items
activity_example_raw = pd.read_sql("SELECT xml FROM xml WHERE name='Accessibility of eLearning'",
                                   con=xml_db.conn).loc[0, "xml"]

# Parse the XML into an xml object
root = etree.fromstring(activity_example_raw)

## Mining OU-XML Documents for Activity Related Items

To begin with, let's have a table that just has the complete activity records.

We can preview a rendered version of an activity by applying an OU-XML to markdown XLST trasnformation to a shimmed version of the activity XML (the shim provides a way of allowing the transformation rules to be applied to arbitrary OU-XML elements that are not defined on the OU-XML root element).

In [3]:
from xml_utils import ouxml2md

example_activity_xml = root.xpath("//Activity")[1]

# Apply the transformation
md = ouxml2md( example_activity_xml )
md

'\n<!-- #region tags=["style-activity"] -->\n### Activity 2\n\n\n#### Question\n\nIn Activity 1 you created a list of types of interaction that might be found in a set of your eLearning materials. Now it is time to [revisit that list](#act1). Did the materials in this course prompt you to think of other interactions you had missed? What about the ways in which disabled students might access your materials &#8211; add to your list any that you may have initially not thought about. Finally, consider ways in which those issues might be removed or avoided, now that you know a little more about how to create accessible eLearning.\n\n<!-- #endregion -->\n'

Render the result:

In [4]:
from IPython.display import Markdown

# Hack because Sphinx gets upset rendering mismatched headers
md = md.replace("###", "HEADER: ")
Markdown(f"<div style='background:lightblue'>\n{md}\n</div>")

<div style='background:lightblue'>

<!-- #region tags=["style-activity"] -->
HEADER:  Activity 2


HEADER: # Question

In Activity 1 you created a list of types of interaction that might be found in a set of your eLearning materials. Now it is time to [revisit that list](#act1). Did the materials in this course prompt you to think of other interactions you had missed? What about the ways in which disabled students might access your materials &#8211; add to your list any that you may have initially not thought about. Finally, consider ways in which those issues might be removed or avoided, now that you know a little more about how to create accessible eLearning.

<!-- #endregion -->

</div>

To keep the size(s) of the databases we are working with relatively small, (<100MB and we can store them in a GitHub repository and serve them from GitHub Pages), we can create a new fatbase for our activities table.

The database table itself can be quite simple, athough note that as well as including the raw XML for the activity, we include a reference to the activity based on its *path* within the unit.

In [5]:
all_activities_raw = db["activities_raw"]
all_activities_raw.drop(ignore=True)
all_activities_raw.create({
    "type": str,
    "activity": str,
    "id": str, # id of unit
    "_path": str
}, pk = ("id", "_path"))

# Enable full text search
# This creates an extra virtual table to support the full text search
db[f"{all_activities_raw.name}_fts"].drop(ignore=True)
db[all_activities_raw.name].enable_fts(["activity",
                                        "id"], create_triggers=True)

<Table activities_raw (type, activity, id, _path)>

Create a simple function to generate the base records for activities, including the path to each activity within an OU-XML dcoument.

In [6]:
from xml_utils import flatten

def get_raw_activity_items(root, typ="Activity", doc_id=None):
    """Get all activities from an OU-XML XML object."""
    
    activities = root.xpath(f'//{typ}')

    activity_list_raw = []
    for activity in activities:
        # Don't store an empty element
        if not flatten(activity):
            continue
    
        # Get path to activity within unit
        tree = etree.ElementTree(activity)
        _path = tree.getpath(activity)

        activity_dict = {"activity": unpack(activity),
                         "type": typ.lower(),
                         "id": doc_id, "_path": _path}

        activity_list_raw.append( activity_dict )

    return activity_list_raw

Let's see how it looks:

In [7]:
get_raw_activity_items(root)

[{'activity': b'<Activity xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" id="act1"><Heading>Activity 1</Heading><Question><NumberedList><ListItem>Identify some eLearning materials you are familiar with. You may have written them, taught them, or even studied them (you can think about this course if you have no examples of your own).</ListItem><ListItem>Identify the different elements of the materials &#8211; there will almost certainly be text, but are there images, videos, forms or boxes to enter text, drag-and-drop exercises, quizzes etc.? List all of the different types of element.</ListItem><ListItem>For each item on the list try to identify where students with particular disabilities might experience difficulties, and try to suggest possible ways (if you can think of any) that these barriers may be removed. It does not matter if your list is incomplete, or if you cannot think of solutions to some of your identified issues. We will revisit this task later, after you have wor

We can now populate the database table by scraping activities from all the OpenLearn OU-XML documents.

We will find it convenient to do this via a function:

In [8]:
from xml_utils import create_id

def all_activity_items_to_db(xml_db, activity_db, typ="Activity"):
    """Parse all OUXML docs and extract specified activity type elements into db."""
    
    for row in xml_db.query("""SELECT * FROM xml;"""):
        _root = etree.fromstring(row["xml"])

        doc_id = create_id( (row["code"], row["name"], ) )
        raw_activity_items = get_raw_activity_items(_root, typ=typ,
                                                    doc_id=doc_id)

        # Add the items to the database  
        activity_db[all_activities_raw.name].insert_all(raw_activity_items)

Now we can populate the database with mined `Activity` elements:

In [9]:
all_activity_items_to_db(xml_db, db, typ="Activity")

Let's see how many activities we have stored:

In [10]:
pd.read_sql("SELECT COUNT(*) AS num_records FROM activities_raw",
            con=db.conn)

,num_records
0,5436


And how about a full-text search over the raw XML?

In [11]:
from xml_utils import fts

example_fts_search = fts(db, "activities_raw", "chemical equation")
example_fts_search

,activity,id
0,b'<Activity>\n <Heading>Act...,904a100e4d41cf1a696b547eec1b2f625fc5bd78
1,"b'<Activity xmlns:xsi=""http://www.w3.org/2001/...",f36ae6f445e1c39925fa84d3e188af9ed7c15fdc


That's not so easy to interpret, as is, so how about if we render the activities in our results list?

In [12]:
md_out = []
for _activity_xml in example_fts_search["activity"]:
    _md = ouxml2md( _activity_xml )
    
    # Hack because Sphinx renders errors
    _md = _md.replace("###", "HEADER: ")
    _md = _md.replace("####", "SUBHEADER: ")
    md_out.append(_md)

Markdown(f"<div style='background:lightblue'>\n{'<hr/>'.join(md_out)}\n</div>")

<div style='background:lightblue'>

<!-- #region tags=["style-activity"] -->
HEADER:  Activity 11 Reaction components
__Timing: Allow about 10 minutes__
<!--
            Heading:
            Part 1-->

HEADER: # Question

Enter the components represented by x and y that complete the formula below.
$$x + CO_2 \rightleftharpoons H_2CO_3 \rightleftharpoons y + HCO_3 \ ^-$$
There are superscript and subscript buttons in the formatting bar. Make sure to use these to enter the correct chemical formula, including the associated positive and negative charges:

x = 

y = 


HEADER: # Answer
$$x = H_2O$$$$y = H^+$$
giving:
$$H_2O + CO_2 \rightleftharpoons H_2CO_3 \rightleftharpoons H^+ + HCO_3 \ ^-$$<!--
            Heading:
            Part 2-->

HEADER: # Question

Using the completed formula above, what will happen to levels of H<sup>+</sup> in the brain as CO<sub>2</sub>-rich blood reaches the medulla?

levels of H<sup>+</sup> will increase

levels of H<sup>+</sup> will decrease

levels of H<sup>+</sup> will stay the same


HEADER: # Answer

Adding more CO<sub>2</sub> will increase the production of H<sup>+</sup> and HCO<sub>3</sub><sup>&#8722;</sup>. Increased H<sup>+</sup> will make the tissue more acidic, meaning that the pH will decrease.

<!-- #endregion -->
<hr/>
<!-- #region tags=["style-activity"] -->
HEADER:  Question 9


HEADER: # Question

How much energy could be obtained from 1 kg of hydrogen (a) if it were to undergo nuclear fusion in the interior of a star, (b) if it were to spiral into a black hole? Would you expect to get more energy if it were to chemically burn in an oxygen atmosphere?


HEADER: # Answer

A mass *m* has a rest energy of *mc*<sup>2</sup>.

(a) If 1 kg of hydrogen were to undergo nuclear fusion to produce helium, the energy liberated would be 0.007 (i.e. 0.7%) of its rest energy:


![
                figure
                https://www.open.edu/openlearn/ocw/pluginfile.php/66727/mod_oucontent/oucontent/469/s282_2_ue015i.gif](https://www.open.edu/openlearn/ocw/pluginfile.php/66727/mod_oucontent/oucontent/469/s282_2_ue015i.gif)

(b) If 1 kg of hydrogen were to fall into a black hole, the energy liberated would be approximately

0.1*mc*<sup>2</sup> = 0.1 &#215; 1 &#215; (3 &#215; 10<sup>8</sup>ms<sup>&#8722;1</sup>)<sup>2</sup> J = 9 &#215; 10<sup>15</sup> J.

You would expect much *less* energy from the chemical reaction.

<!-- #endregion -->

</div>

### Adding Other Activity Types to the Database

We can also add other activity types (`<ITQ>`, `<SAQ>` and `<Exercise>`) to the raw activity database:

In [13]:
all_activity_items_to_db(xml_db, db, typ="ITQ")
all_activity_items_to_db(xml_db, db, typ="SAQ")
all_activity_items_to_db(xml_db, db, typ="Exercise")

Now we can search for ITQ questions as well:

In [14]:
example_itqs = pd.read_sql("SELECT * FROM activities_raw WHERE type='itq' \
                                AND activity LIKE '%molecule%compound%'",
                           con=db.conn)

md_out = []
for _itq_xml in example_itqs['activity']:
    _md = ouxml2md( _itq_xml )
    # Hack because Sphinx renders errors
    _md = _md.replace("####", "HEADER: ")
    md_out.append(_md)

Markdown(f"<div style='background:lightblue'>\n{'<hr/>'.join(md_out)}\n</div>")

<div style='background:lightblue'>
<!--ITQ-->

HEADER:  Question

Are the following compounds molecular or non-molecular? Chlorine (Cl<sub>2</sub>) and sodium chloride (NaCl).


HEADER:  Answer

* 
Chlorine (Cl<sub>2</sub>) is a molecular compound &#8211; it is made up of discrete Cl<sub>2</sub> molecules.


* 
Sodium chloride (NaCl) a non-molecular compound, is made up of Na<sup>+</sup> and Cl<sup>- </sup>ions.

<!--ENDITQ--><hr/><!--ITQ-->

HEADER:  Question

Is Pt2+ a hard or soft acid? What types of molecule or ion in the bloodstream might react with cisplatin before it gets a chance to cross the cell membrane?


HEADER:  Answer

There are many species present in blood, including sugars, salt, proteins, oxygen and, of course, water. Pt2+ is a soft acid, so soft bases pose the greatest threat, also those species that are in the greatest concentration. Thus sulfur-containing compounds, such as cysteine, might react with cisplatin, as might water.
<!--ENDITQ-->
</div>

*It might be worth having the raw `Activity`, `Exercise`, `ITQ` and `SAQ` items in their own databases so that we can then run a full-text search over the raw XML for just those activity types. (I donlt think we can run facetted queries within the SQLite full-text search?*

## Decomposing Activities Into Component Parts

Activity-like objects are made up of several "simple" and more structurd componenents. All activity types must define a `<Question>` component, but there is some freedom in the choice of "response" element. In addition, multipart responses may be defined.

The strategy followed below is to split out the "simple" `<Answer>` and `<Discussion>` elements ito their own columns. For `<Multipart>` responses, each `<Part>` has its own entry in the `activities` table, with a `multipart` flag set.

*TO DO: a "part-sort-order" number should also be included in the table.*

The more complex `<MediaContent>` and `<Interaction>` elements are not currently handled in a meanigful way in the activity context. (Media items are handled separately in their own mined table, and a foreigh key reference should be made into that table. The interaction element is currently included in the activities table as stringified XML.)

*TO DO: handle interaction*

*TO DO: handle media item reference.*

In [15]:
import secrets
        
def parse_activity_item(activity, _path=None, key=""):
    """Parse activity element.
       The key is a unit id that lets us create a unique interaction table FK."""
    def _delist(x):
        return x[0] if x else x
    def _tidy(x):
        return unpack(x).decode().strip() if len(x) else None
    
    if _path is None:
        tree = etree.ElementTree(activity)
        _path = tree.getpath(activity)

    key = key if key else secrets.token_hex(16)
        
    # Need to consider Multipart
    _flat = flatten(activity)
    
    if _flat:
        a_multipart=[]
        a_heading = activity.xpath("Heading")
        a_heading = flatten(a_heading[0]) if a_heading else None
        a_timing = activity.xpath("Timing")
        a_timing = flatten(a_timing[0]) if a_timing else None
        # Should we perhaps parse the question to markdown?
        a_question = _tidy(_delist(activity.xpath("Question")))
        a_answer = _tidy(_delist(activity.xpath("Answer")))
        
        """
        # The interaction type is a complex element that should be included
        # in an interaction table with a shared reference id generated from
        # the unit id and the path to the element
        _interaction = _tidy(_delist(activity.xpath("Interaction")))
        if _interaction:
            interaction_id = create_id( (key, _path) )
            a_interaction = interaction_id
        else:
            a_interaction = None
        """
        a_discussion = _tidy(_delist(activity.xpath("Discussion")))
        
        # TO DO - for now, explicitly capture the interaction
        a_interaction = _tidy(_delist(activity.xpath("Interaction")))
        
        # TO DO: activity.xpath("MediaElement") as a FK relation?
        
        if activity.xpath("Multipart"):
            for p in activity.xpath("Multipart/Part"):
                a_multipart.extend(parse_activity_item(p))

        if not a_multipart:
            # Also return the multipart status and the multipart name and timing
            return [(a_heading, a_timing, a_question, a_interaction, a_answer,
                     a_discussion, _path,
                    True if a_multipart else False, _path)]
        else:
            return [(m[0], m[1], m[2], m[3], m[4], m[5], m[6],
                    True if a_multipart else False, _path) for m in a_multipart if m]

    return []

Let's see how that works:

In [16]:
parse_activity_item( root.xpath("//Activity")[0])

[('Activity 1',
  None,
  '<Question xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><NumberedList><ListItem>Identify some eLearning materials you are familiar with. You may have written them, taught them, or even studied them (you can think about this course if you have no examples of your own).</ListItem><ListItem>Identify the different elements of the materials &#8211; there will almost certainly be text, but are there images, videos, forms or boxes to enter text, drag-and-drop exercises, quizzes etc.? List all of the different types of element.</ListItem><ListItem>For each item on the list try to identify where students with particular disabilities might experience difficulties, and try to suggest possible ways (if you can think of any) that these barriers may be removed. It does not matter if your list is incomplete, or if you cannot think of solutions to some of your identified issues. We will revisit this task later, after you have worked through more of the course materia

*TO DO* - the `<Interaction>` element takes several forms: `<SingleChoice>`, `<Matching>`, `<FreeResponse>`, `<VoiceRecorder>`, `<MultipleChoice>`.

As such, it makes sense to record interaction components in a table of their own, referenced to the parent activity element etc. TO DO

We can also create a table that splits the activity elements out into component parts:

In [17]:
all_activities_tbl = db["activities"]
all_activities_tbl.drop(ignore=True)
all_activities_tbl.create({
    "type": str,
    "heading": str,
    "timing": str,
    "question": str,
    "answer": int,
    "discussion": str,
    "interaction": str,
    "multipart": bool,
    "id": str, # id of unit
    "_path": str
})
# For the pk, we need to be able to account for multipart elements
# Currently, each part in multipart element is an entry in this table

# Enable full text search
# This creates an extra virtual table (media_fts) to support the full text search
db[f"{all_activities_tbl.name}_fts"].drop(ignore=True)
db[all_activities_tbl.name].enable_fts(["heading", "question", "interaction",
                                                 "answer", "discussion", "id"],
                                                create_triggers=True)

<Table activities (type, heading, timing, question, answer, discussion, interaction, multipart, id, _path)>

Create a simple function to grab all the activities associated with a particular unit:

In [18]:
def get_activity_items(root, typ="Activity", _id=""):
    """Extract activity items from an OU-XML XML object."""
    activities = root.xpath(f'//{typ}')
    
    activity_list_raw = []
    activity_list = []
    for activity in activities:
            
        # Get path to activity within unit
        tree = etree.ElementTree(activity)
        _path = tree.getpath(activity)
    
        activity_list.extend(parse_activity_item(activity, _path))
        activity_list_raw.append( {"activity": unpack(activity), "id": _id,
                                   "type":typ.lower(), "_path": _path} )

    return activity_list_raw, activity_list

Create a function to scan the OpenLearn units for various type of activity:

In [19]:
def add_activities_to_db(typ="Activity"):
    """Add activity type elements to the database."""
    for row in xml_db.query("""SELECT * FROM xml;"""):
        _root = etree.fromstring(row["xml"])
        raw_activity_items, activity_items = get_activity_items(_root,
                                                                typ=typ,
                                                                _id=row["id"])

        # From the list of activity items,
        # create a list of dict items we can add to the database
        activity_item_dicts = [{"heading": a[0], "timing": a[1] , "question": a[2],
                                "interaction": a[3], "answer": a[4],
                                "discussion": a[5], "_path": a[6],
                                "type": typ.lower(),
                                "id": row["id"]} for a in activity_items if a[2] ]

        # Add items to the database
        db[all_activities_tbl.name].insert_all(activity_item_dicts)

We can now parse the documents for `Activity` type elements:

In [20]:
add_activities_to_db(typ="Activity")

How does that look?

In [21]:
pd.read_sql("SELECT * FROM activities LIMIT 3", con=db.conn)

,type,heading,timing,question,answer,discussion,interaction,multipart,id,_path
0,activity,Activity 1 Looking at different technologies,Allow approximately 15 minutes,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...",None,"<Discussion xmlns:xsi=""http://www.w3.org/2001/...","<Interaction xmlns:xsi=""http://www.w3.org/2001...",None,1f194525072f4358f7639c471ee5289665d50a3f,/Item/Unit/Session[1]/Activity
1,activity,Activity 2 Types of body language,Allow approximately 20 minutes,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...",None,"<Discussion xmlns:xsi=""http://www.w3.org/2001/...",None,None,1f194525072f4358f7639c471ee5289665d50a3f,/Item/Unit/Session[5]/Activity
2,activity,Activity 3 Translating emoji language,Allow approximately 15 minutes,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...",None,"<Discussion xmlns:xsi=""http://www.w3.org/2001/...",None,None,1f194525072f4358f7639c471ee5289665d50a3f,/Item/Unit/Session[6]/Activity


And for the raw table?

In [22]:
pd.read_sql("SELECT * FROM activities_raw LIMIT 3", con=db.conn)

,type,activity,id,_path
0,activity,"b'<Activity xmlns:xsi=""http://www.w3.org/2001/...",1f194525072f4358f7639c471ee5289665d50a3f,/Item/Unit/Session[1]/Activity
1,activity,"b'<Activity xmlns:xsi=""http://www.w3.org/2001/...",1f194525072f4358f7639c471ee5289665d50a3f,/Item/Unit/Session[5]/Activity
2,activity,"b'<Activity xmlns:xsi=""http://www.w3.org/2001/...",1f194525072f4358f7639c471ee5289665d50a3f,/Item/Unit/Session[6]/Activity


## Parsing Other Activity Types

An ITQ element can be parsed in the same way as `<Activity>` element, as previously described:

In [23]:
add_activities_to_db(typ="ITQ")

pd.read_sql("SELECT * FROM activities WHERE type='itq' LIMIT 3", con=db.conn)

,type,heading,timing,question,answer,discussion,interaction,multipart,id,_path
0,itq,None,None,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...","<Answer xmlns:xsi=""http://www.w3.org/2001/XMLS...",None,None,None,6bff78840be5165329dda278418bbbd54c909047,/Item/Unit/Session[1]/Section[2]/ITQ
1,itq,None,None,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...","<Answer xmlns:xsi=""http://www.w3.org/2001/XMLS...",None,None,None,6bff78840be5165329dda278418bbbd54c909047,/Item/Unit/Session[1]/Section[3]/SubSection[1]...
2,itq,None,None,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...","<Answer xmlns:xsi=""http://www.w3.org/2001/XMLS...",None,None,None,6bff78840be5165329dda278418bbbd54c909047,/Item/Unit/Session[1]/Section[3]/SubSection[6]...


How about exercises?

In [24]:
add_activities_to_db(typ="Exercise")

pd.read_sql("SELECT * FROM activities WHERE type='exercise' LIMIT 3", con=db.conn)

,type,heading,timing,question,answer,discussion,interaction,multipart,id,_path


Or SAQs?

In [25]:
add_activities_to_db(typ="SAQ")

pd.read_sql("SELECT * FROM activities WHERE type='saq' LIMIT 3", con=db.conn)

,type,heading,timing,question,answer,discussion,interaction,multipart,id,_path
0,saq,None,None,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...","<Answer xmlns:xsi=""http://www.w3.org/2001/XMLS...",None,None,None,6c5ba9c60fa29f546a71ba94565a6f62f1eae0db,/Item/Unit[2]/Session[2]/Section/SAQ
1,saq,None,None,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...",None,"<Discussion xmlns:xsi=""http://www.w3.org/2001/...",None,None,6c5ba9c60fa29f546a71ba94565a6f62f1eae0db,/Item/Unit[2]/Session[3]/Section[2]/SAQ[1]
2,saq,None,None,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...","<Answer xmlns:xsi=""http://www.w3.org/2001/XMLS...",None,None,None,6c5ba9c60fa29f546a71ba94565a6f62f1eae0db,/Item/Unit[2]/Session[3]/Section[2]/SAQ[2]


We should also be able to run full-text search questions over all the activity types:

In [26]:
fts(db, "activities", "chemical equation")

,heading,question,interaction,answer,discussion,id
0,Part 1,<Question>\n <P...,<Interaction>\n ...,<Answer>\n <Equ...,None,904a100e4d41cf1a696b547eec1b2f625fc5bd78
1,Question 9,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...",None,"<Answer xmlns:xsi=""http://www.w3.org/2001/XMLS...",None,f36ae6f445e1c39925fa84d3e188af9ed7c15fdc
2,None,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...",None,"<Answer xmlns:xsi=""http://www.w3.org/2001/XMLS...",None,884164a46f4066c6b26894c812484c74ab2e8531
3,None,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...",None,"<Answer xmlns:xsi=""http://www.w3.org/2001/XMLS...",None,884164a46f4066c6b26894c812484c74ab2e8531
4,None,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...",None,"<Answer xmlns:xsi=""http://www.w3.org/2001/XMLS...",None,884164a46f4066c6b26894c812484c74ab2e8531
5,None,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...",None,"<Answer xmlns:xsi=""http://www.w3.org/2001/XMLS...",None,884164a46f4066c6b26894c812484c74ab2e8531
6,None,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...",None,"<Answer xmlns:xsi=""http://www.w3.org/2001/XMLS...",None,884164a46f4066c6b26894c812484c74ab2e8531
7,SAQ 5,"<Question xmlns:xsi=""http://www.w3.org/2001/XM...",None,"<Answer xmlns:xsi=""http://www.w3.org/2001/XMLS...",None,a82aa8fe5c90c02095eefb3e7d998efbbc1949c8


We really need a better way to render these results...

Parsing the activity into markdown would be one way...